In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.optimizers import Adam

import seaborn as sns
sns.set()

Using TensorFlow backend.


# Data

In [2]:
data = pd.read_csv('D:/IITM/DeepCache/Medysin/mediSynDataset_x2_O3488.csv')
data.head()

,object_ID,request_time
0,2033,86416.158584
1,1280,86419.381316
2,3482,86426.764935
3,2764,86429.048406
4,2698,86430.680023


In [3]:
data = np.array(data)
data.shape

(218855802, 2)

In [4]:
np.min(data[:,0]),np.max(data[:,0])

(1.0, 3488.0)

In [5]:
np.min(data[:,1]),np.max(data[:,1])

(86416.15858448479, 17343055.15509677)

In [6]:
data[:,1] = data[:,1] - np.min(data[:,1])

In [7]:
#to find requests in 1h
i = 2000
while data[i,1] < 3600:
    i += 1
print(i,data[i,1])

2281 3603.9286053070246


In [126]:
def prob(i):
    j = i + 2281
    interval = 0
    while interval < 3600:
        j -= 1
        interval = data[i+2281,1] - data[j,1]
    count = np.zeros(3488,dtype=int)
    for k in range(j,i+2281):
        t = int(data[k,0])
        count[t] += 1 
    return count/(i-j+2281)

In [133]:
x = len(data) - 2280
x*3488/1000000000

763.361084736